<a href="https://colab.research.google.com/github/thirumalvenkat/Sample/blob/master/grp2_capstone_gl_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notebook is created and owned by Capstone Group 2 of Great Learnings Class of August2019/AI/ML. Confidential and cannot be shared with anyone and rights owned by the following members

1.   Thirumalvalavan Venkatesan thirumal.valavan@gmail.com 
2.   Sundaresan Mooorthy 
1.   Vivek Suresh
2.   Muthu Govindraraj
1.   Riaz
2.   Prasanna Dravidamani

## 01 Google Cloud Environment Setup

In [ ]:
%env PROJECT_ID = stately-planet-181816
#Set the Project Id
!gcloud config set project $PROJECT_ID

#Authenticate the project
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

env: PROJECT_ID=stately-planet-181816
Updated property [core/project].


In [ ]:
#Set the variables for bucket and Zone
%env BUCKET_NAME = grp2-capstone-gl
%env REGION = us-central1

## 02 Import required packages 

In [ ]:
from tensorflow import keras
from keras.preprocessing import image as keras_image
import os
import numpy as np
from IPython.display import display, Image
import PIL
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import time
import pickle
from datetime import timedelta
from sklearn.model_selection import train_test_split
# All Keras Packages and backends
#import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
#from keras_yolo3.yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
#from keras_yolo3.yolo3.utils import get_random_data
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from keras.callbacks import TensorBoard

## 03 Mount the Google Cloud Storage Bucket using GCSFUSE

In [ ]:
!gsutil ls gs://$BUCKET_NAME

In [ ]:
!sudo apt-get update

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -

In [ ]:
!apt -qq update
!apt -qq install gcsfuse

In [ ]:
!mkdir -p data

In [ ]:
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 grp2-capstone-gl data

In [ ]:
!ls -ltr /content/data

## 04 Exploratory Data Analysis 
Read the Logo27 datasets from GCS bucket

In [ ]:
!ls -ltr /content/data/flickr_logos_27_dataset_images/ | wc

In [ ]:
train_data = "/content/data/flickr_logos_27_dataset_images/"

In [ ]:
n_train = len(os.listdir(train_data))
sample_idxs = np.random.choice(n_train, 5, replace=False)

#First Time Trying Ipython to Display the Images. I will do it through other means also. 
for idx in sample_idxs:
  filename = os.listdir(train_data)[idx]
  image = Image(os.path.join(train_data, filename))
  display(image)

In [ ]:
widths, heights = [], []
width_min, width_max = np.inf, -np.inf
height_min, height_max = np.inf, -np.inf
for filename in os.listdir(train_data):
  image = PIL.Image.open(os.path.join(train_data, filename))
  widths.append(image.size[0]) # width
  heights.append(image.size[1]) # height
# Print out stats for viewing pleasure.
print('Training data average (width, height): ', (np.mean(widths), np.mean(heights)))
print('Training data (minimum width, maximum width): ', (np.min(widths), np.max(widths)))
print('Training data (minimum height, maximum height): ', (np.min(heights), np.max(heights)))
print('Training data (STD of widths, STD of heights): ', (np.std(widths), np.std(heights)))

## 05 Labels Validation and EDA

In [ ]:
colnames=["filename", "label","subset","x1","y1","x2","y2"]
labels_directory="/content/data/flickr_logos_27_dataset/"
labels_file="flickr_logos_27_dataset_training_set_annotation.txt"
logo_labels = pd.read_csv("{}/{}".format(labels_directory,labels_file),header=0,names=colnames, sep=r"\s+")

In [ ]:
logo_labels.head()

In [ ]:
logo_labels[(logo_labels['filename'] == '4763210295.jpg') & (logo_labels['subset'] == 1)]
#logo_labels[(logo_labels['subset'] == 1)]

In [ ]:
temp_file="4763210295.jpg"
image = Image(os.path.join(train_data, temp_file))
display(image)

In [ ]:
logo_labels.describe()

In [ ]:
label_files_grp = logo_labels.groupby('filename').count()
label_files_count = label_files_grp['label'].count()

In [ ]:
num_logos = logo_labels['label'].value_counts()
num_logos_gp = logo_labels.groupby('label').count()
num_logos_count = num_logos.count()

In [ ]:
print("Number of Logos Count:",num_logos)
print("Number of Logos in the Dataset:",num_logos_count)
print("Total Number of files:",label_files_count)

In [ ]:
hist = logo_labels['label'].hist(figsize=(16, 8), bins=120, xrot=90, xlabelsize=8)

In [ ]:
le = LabelEncoder()
logo_labels['label'] = le.fit_transform(logo_labels['label'])
train_labels = keras.utils.to_categorical(logo_labels['label'], num_classes=num_logos_count)

## 06 Prepare Training Dataset

In [ ]:
#Initialize all the variables
img_rows = 128

In [ ]:
def load_images(input_size,file_name,tgt_size,labels):
  all_images=[]
  # Start time
  start_time = time.time()
  for i in range(input_size):
    # Set the Path
    path = labels.iloc[i].filename
    path = os.path.join(train_data,path)
    #Read the files usign Keras
    img  = keras_image.load_img(path,target_size=(tgt_size,tgt_size))
    #Resize the images
    img_array = keras_image.img_to_array(img)
    #Just Print the status for monitoring
    if (i%200) == 0:
      print(i)
    all_images.append(img_array)
    #End For Loop  
  end_time = time.time()
  # Difference between start and end-times.
  time_dif = end_time - start_time
  # Print the time-usage.
  pickle.dump(all_images, open( file_name + '.p', "wb" ) )
  print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))
  return all_images

In [ ]:
#train_images = load_images(n_train,'train_load_images',img_rows,logo_labels)

In [ ]:
#!mkdir /content/data/artefacts
#!mv 'train_load_images.p' /content/data/artefacts/train_load_new_images.p

In [ ]:
#load TRAIN
train_images = pickle.load(open("/content/data/artefacts/train_load_new_images.p", "rb" ))

In [ ]:
print("Shape of Train Images:",np.shape(train_images))
print("Shape of Train Labels:",np.shape(train_labels))

In [ ]:
train_f_images = np.array(train_images)
train_f_images = train_f_images/255.0

## 07 Train and Validation Split
Might not be needed if we figure out the test datasets separately

In [ ]:
trainx_images,val_images, trainx_labels, val_labels = train_test_split(train_f_images, train_labels, test_size = 0.2, random_state=10)